In [ ]:
DECLARE payoff_search_start TIMESTAMP DEFAULT TIMESTAMP("2024-07-01 00:00:00 UTC");
DECLARE payoff_search_end   TIMESTAMP DEFAULT TIMESTAMP("2025-07-30 00:00:00 UTC");
DECLARE usdc_addr STRING DEFAULT "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48";
DECLARE usdt_addr STRING DEFAULT "0xdac17f958d2ee523a2206206994597c13d831ec7";

WITH 
-- 1. LOAD DETECTED ATTEMPTS
attack_attempts AS (
    SELECT 
        attack_tx_hash,
        attack_ts,
        victim_address,
        attacker_address,
        token_symbol, 
        attack_type
    FROM `cs356-478822.eth_vlad_query.vald_query_attacks`
    WHERE attack_ts BETWEEN payoff_search_start AND TIMESTAMP("2025-07-01 00:00:00 UTC")
),

-- 2. FIND CANDIDATE PAYOFFS (Victim -> Attacker)
raw_payoffs AS (
    SELECT 
        transaction_hash AS payoff_tx_hash,
        block_timestamp AS payoff_ts,
        LOWER(from_address) AS victim_address,
        LOWER(to_address) AS attacker_address,
        SAFE_CAST(quantity AS BIGNUMERIC) / 1e6 AS payoff_value_usd,
        CASE WHEN LOWER(address) = usdc_addr THEN 'USDC' ELSE 'USDT' END AS token_symbol
    FROM bigquery-public-data.goog_blockchain_ethereum_mainnet_us.token_transfers
    WHERE 
        block_timestamp BETWEEN payoff_search_start AND payoff_search_end
        AND LOWER(address) IN (usdc_addr, usdt_addr)
        AND LOWER(from_address) IN (SELECT DISTINCT victim_address FROM attack_attempts)
        AND LOWER(to_address) IN (SELECT DISTINCT attacker_address FROM attack_attempts)
),

-- 3. CONFIRM THE ATTACK CHAIN
confirmed_payoffs AS (
    SELECT 
        p.payoff_tx_hash,
        p.payoff_ts,
        p.payoff_value_usd,
        p.token_symbol,
        a.attack_tx_hash,
        a.attack_ts,
        a.attack_type,
        p.victim_address, 
        TIMESTAMP_DIFF(p.payoff_ts, a.attack_ts, HOUR) AS hours_until_payoff
    FROM raw_payoffs p
    JOIN attack_attempts a
        ON p.victim_address = a.victim_address 
        AND p.attacker_address = a.attacker_address
    WHERE 
        p.payoff_ts > a.attack_ts
)

-- 4. FINAL REPORT
SELECT 
    payoff_tx_hash,
    payoff_ts,
    token_symbol,
    victim_address,
    attack_type,
    attack_tx_hash, -- This remains the single "most recent" attack hash
    
    -- NEW COLUMN: Collects ALL attack hashes for this payoff into an array
    ARRAY_AGG(attack_tx_hash) OVER(
        PARTITION BY payoff_tx_hash
    ) AS attack_attempts,

    payoff_value_usd,
    hours_until_payoff,
    CONCAT('https://etherscan.io/tx/', payoff_tx_hash) as etherscan_link
FROM confirmed_payoffs
WHERE payoff_value_usd > 10 

-- DEDUPLICATION
QUALIFY ROW_NUMBER() OVER(
    PARTITION BY payoff_tx_hash 
    ORDER BY attack_ts DESC
) = 1

ORDER BY payoff_value_usd DESC

## Arbitrum 

In [ ]:
%%bigquery
-- 1. CONFIGURATION
DECLARE payoff_search_start TIMESTAMP DEFAULT TIMESTAMP("2024-07-01 00:00:00 UTC");
DECLARE payoff_search_end   TIMESTAMP DEFAULT TIMESTAMP("2025-07-30 00:00:00 UTC");

-- ARBITRUM TOKEN ADDRESSES
DECLARE usdc_native  STRING DEFAULT "0xaf88d065e77c8cc2239327c5edb3a432268e5831";
DECLARE usdc_bridged STRING DEFAULT "0xff970a61a04b1ca14834a43f5de4533ebddb5cc8";
DECLARE usdt_addr    STRING DEFAULT "0xfd086bc7cd5c481dcc9c85ebe478a1c0b69fcbb9";

-- 0. HELPER FUNCTION (Required for Arbitrum Log Decoding)
CREATE TEMP FUNCTION HexToBigNumeric(x STRING)
RETURNS BIGNUMERIC LANGUAGE js AS """
  if (!x || x === '0x') return 0;
  return BigInt(x).toString();
""";

WITH 
-- 1. LOAD DETECTED ATTEMPTS (Arbitrum Table)
attack_attempts AS (
    SELECT 
        attack_tx_hash,
        attack_ts,
        victim_address,
        attacker_address,
        attack_type
    FROM `cs356-480905.arbitrum_vlad_query.arbitrum_attacks` -- Put your attack table 
    WHERE attack_ts BETWEEN payoff_search_start AND TIMESTAMP("2025-07-01 00:00:00 UTC")
),

-- 2. FIND CANDIDATE PAYOFFS (Decoded from Arbitrum Logs)
raw_payoffs AS (
    SELECT 
        transaction_hash AS payoff_tx_hash,
        block_timestamp AS payoff_ts,
        
        -- Decode FROM (Topic 1)
        LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27))) AS victim_address,
        -- Decode TO (Topic 2)
        LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27))) AS attacker_address,
        
        -- Decode Value (Data field) / 1e6 for USDC/USDT
        HexToBigNumeric(data) / 1e6 AS payoff_value_usd,
        
        -- Identify Token Symbol based on Contract Address
        CASE 
            WHEN LOWER(address) = usdc_native  THEN "USDC.e" -- or USDC Native
            WHEN LOWER(address) = usdc_bridged THEN "USDC"   -- or USDC Bridged
            ELSE "USDT"
        END AS token_symbol

    FROM `bigquery-public-data.goog_blockchain_arbitrum_one_us.logs`
    WHERE 
        block_timestamp BETWEEN payoff_search_start AND payoff_search_end
        -- Filter for Transfer Events (0xddf2...)
        AND topics[SAFE_OFFSET(0)] = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
        AND address IN (usdc_native, usdc_bridged, usdt_addr)
        
        -- Optimization: Pre-filter using known addresses from attack table
        AND LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27))) IN (SELECT DISTINCT victim_address FROM attack_attempts)
        AND LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27))) IN (SELECT DISTINCT attacker_address FROM attack_attempts)
),

-- 3. CONFIRM THE ATTACK CHAIN
confirmed_payoffs AS (
    SELECT 
        p.payoff_tx_hash,
        p.payoff_ts,
        p.payoff_value_usd,
        p.token_symbol,
        a.attack_tx_hash,
        a.attack_ts,
        a.attack_type,
        p.victim_address, 
        TIMESTAMP_DIFF(p.payoff_ts, a.attack_ts, HOUR) AS hours_until_payoff
    FROM raw_payoffs p
    JOIN attack_attempts a
        ON p.victim_address = a.victim_address 
        AND p.attacker_address = a.attacker_address
    WHERE 
        -- Confirm Payoff happened AFTER the Attack
        p.payoff_ts > a.attack_ts
)

-- 4. FINAL REPORT
SELECT 
    payoff_tx_hash,
    payoff_ts,
    token_symbol,
    victim_address,
    attack_type,
    attack_tx_hash, -- The single "winning" attack hash
    
    -- ARRAY of ALL attempts for this payoff
    ARRAY_AGG(attack_tx_hash) OVER(
        PARTITION BY payoff_tx_hash
    ) AS attack_attempts,

    payoff_value_usd,
    hours_until_payoff,
    -- Updated to Arbiscan
    CONCAT('https://arbiscan.io/tx/', payoff_tx_hash) as arbiscan_link
FROM confirmed_payoffs
WHERE payoff_value_usd > 10 

-- DEDUPLICATION
QUALIFY ROW_NUMBER() OVER(
    PARTITION BY payoff_tx_hash 
    ORDER BY attack_ts DESC
) = 1

ORDER BY payoff_value_usd DESC

Optimism

In [ ]:
%% bigquery
-- SELECT  FROM `cs356-478822.optimism_vlad_query.optimism_attacks` LIMIT 1000
-- https://optimistic.etherscan.io/
-- 1. CONFIGURATION
DECLARE payoff_search_start TIMESTAMP DEFAULT TIMESTAMP("2024-07-01 00:00:00 UTC");
DECLARE payoff_search_end   TIMESTAMP DEFAULT TIMESTAMP("2025-07-30 00:00:00 UTC");

-- Optimism TOKEN ADDRESSES
DECLARE usdc_native  STRING DEFAULT "0x0b2c639c533813f4aa9d7837caf62653d097ff85";
DECLARE usdc_bridged STRING DEFAULT "0x7f5c764cbc14f9669b88837ca1490cca17c31607";
DECLARE usdt_addr    STRING DEFAULT "0x94b008aa00579c1307b0ef2c499ad98a8ce58e58";

-- 0. HELPER FUNCTION (Required for Optimism Log Decoding)
CREATE TEMP FUNCTION HexToBigNumeric(x STRING)
RETURNS BIGNUMERIC LANGUAGE js AS """
  if (!x || x === '0x') return 0;
  return BigInt(x).toString();
""";

WITH 
-- 1. LOAD DETECTED ATTEMPTS (Optimism Table)
attack_attempts AS (
    SELECT 
        attack_tx_hash,
        attack_ts,
        victim_address,
        attacker_address,
        attack_type
    FROM `cs356-478822.optimism_vlad_query.optimism_attacks` -- Put your attack table 
    WHERE attack_ts BETWEEN payoff_search_start AND TIMESTAMP("2025-07-01 00:00:00 UTC")
),

-- 2. FIND CANDIDATE PAYOFFS (Decoded from Optimism Logs)
raw_payoffs AS (
    SELECT 
        transaction_hash AS payoff_tx_hash,
        block_timestamp AS payoff_ts,
        
        -- Decode FROM (Topic 1)
        LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27))) AS victim_address,
        -- Decode TO (Topic 2)
        LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27))) AS attacker_address,
        
        -- Decode Value (Data field) / 1e6 for USDC/USDT
        HexToBigNumeric(data) / 1e6 AS payoff_value_usd,
        
        -- Identify Token Symbol based on Contract Address
        CASE 
            WHEN LOWER(address) = usdc_native  THEN "USDC.e" -- or USDC Native
            WHEN LOWER(address) = usdc_bridged THEN "USDC"   -- or USDC Bridged
            ELSE "USDT"
        END AS token_symbol

    FROM `bigquery-public-data.goog_blockchain_optimism_mainnet_us.logs`
    WHERE 
        block_timestamp BETWEEN payoff_search_start AND payoff_search_end
        -- Filter for Transfer Events (0xddf2...)
        AND topics[SAFE_OFFSET(0)] = "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef"
        AND address IN (usdc_native, usdc_bridged, usdt_addr)
        
        -- Optimization: Pre-filter using known addresses from attack table
        AND LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(1)], 27))) IN (SELECT DISTINCT victim_address FROM attack_attempts)
        AND LOWER(CONCAT("0x", SUBSTR(topics[SAFE_OFFSET(2)], 27))) IN (SELECT DISTINCT attacker_address FROM attack_attempts)
),

-- 3. CONFIRM THE ATTACK CHAIN
confirmed_payoffs AS (
    SELECT 
        p.payoff_tx_hash,
        p.payoff_ts,
        p.payoff_value_usd,
        p.token_symbol,
        a.attack_tx_hash,
        a.attack_ts,
        a.attack_type,
        p.victim_address, 
        TIMESTAMP_DIFF(p.payoff_ts, a.attack_ts, HOUR) AS hours_until_payoff
    FROM raw_payoffs p
    JOIN attack_attempts a
        ON p.victim_address = a.victim_address 
        AND p.attacker_address = a.attacker_address
    WHERE 
        -- Confirm Payoff happened AFTER the Attack
        p.payoff_ts > a.attack_ts
)

-- 4. FINAL REPORT
SELECT 
    payoff_tx_hash,
    payoff_ts,
    token_symbol,
    victim_address,
    attack_type,
    attack_tx_hash, -- The single "winning" attack hash
    
    -- ARRAY of ALL attempts for this payoff
    ARRAY_AGG(attack_tx_hash) OVER(
        PARTITION BY payoff_tx_hash
    ) AS attack_attempts,

    payoff_value_usd,
    hours_until_payoff,
    -- Updated to Optimism scan
    CONCAT('https://optimistic.etherscan.io/', payoff_tx_hash) as optimism_link
FROM confirmed_payoffs
WHERE payoff_value_usd > 10 

-- DEDUPLICATION
QUALIFY ROW_NUMBER() OVER(
    PARTITION BY payoff_tx_hash 
    ORDER BY attack_ts DESC
) = 1

ORDER BY payoff_value_usd DESC